# How to remove nodes from a tree structure in Dyalog APL

Let us take a simple example of some XHTML. We are going to remove tags with `class="remove"` but keep their children, lifting all descendants one level in the tree.

The following is stored in a character vector `xhtml`:

In [1]:
nl←⎕UCS 10
xhtml ← '<div class="remove">',nl
xhtml,← '  <h1>Title</h1>',nl
xhtml,← '  <div>',nl
xhtml,← '    <p>nested p in nested div</p>',nl
xhtml,← '  </div>',nl
xhtml,← '  <p>Some text</p>',nl
xhtml,← '</div>',nl
xhtml,← nl
xhtml,← '<div>',nl
xhtml,← '  <p class="remove">',nl
xhtml,← '    Here is text with <strong>bold</strong> tag inside.',nl
xhtml,← '  </p>',nl
xhtml,← '</div>',nl

In [2]:
xhtml

<div class="remove">
 <h1>Title</h1>
 <div>
 <p>nested p in nested div</p>
 </div>
 <p>Some text</p>
</div>

<div>
 <p class="remove">
 Here is text with <strong>bold</strong> tag inside.
 </p>
</div>

First we use `⎕XML` to parse the tree structure into a depth-vector representation.

The result of `⎕XML` is a matrix with columns:
- depth `d` integer vector of node depth in order of a depth-first pre-order traversal.
- XML tag `t` nested vector of character vectors
- value `v` nested vector of character vectors
- attributes `a` nested vector of nested matrices, each with 2-element rows of the attribute key and value
- kind `k` is a numeric vector which indicates whether the row contains an element, child element, character data etc. according to the table in [the `⎕XML` documentation](https://help.dyalog.com/latest/#Language/System%20Functions/xml.htm)

We extract these columns into a vector variable for each column using split-transpose `↓⍉`:

In [4]:
(d t v a k) ← ↓⍉ ⎕XML xhtml

Now we identify the tags to remove.

We want to find nodes with a row in their attributes matrix `'class' 'remove'`. The high-rank version of the *membership* function uses index-of to see whether a cell `⍺` is present as a major cell in `⍵`.

In [5]:
E←{(≢⍵)≥⍵⍳⍺}

In [6]:
⎕←remove←'class' 'remove'∘E¨a

1 0 0 0 0 0 1 0 0 0

Note on faster lookups. The nested array representation returned by `⎕XML` is inefficient. With some overhead at conversion time, we can have a flat representation of each column - also known as an inverted table.

In [19]:
↑¨t v a

┌──────┬──────────────────────┬──────────────┐
│div │ │┌─────┬──────┐│
│h1 │Title ││class│remove││
│div │ │└─────┴──────┘│
│p │nested p in nested div│┌─────┬──────┐│
│p │Some text ││ │ ││
│div │ │└─────┴──────┘│
│p │ │┌─────┬──────┐│
│ │Here is text with ││ │ ││
│strong│bold │└─────┴──────┘│
│ │tag inside. │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││class│remove││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
│ │ │┌─────┬──────┐│
│ │ ││ │ ││
│ │ │└─────┴──────┘│
└──────┴──────────────────────┴──────────────┘

The easiest way to identify descendants is using the parent vector `p`, defined from `d` with the following idiom:

In [7]:
{}2{p[⍵]←⍺[⍺⍸⍵]}⌿⊢∘⊂⌸d⊣p←⍳≢d

The leading `{}` is to ensure that data from the reduction is discarded.

Any top-level node is its own parent.

In [8]:
i←⍳≢d
↑i d p t v

┌───┬─────┬───┬──────────────────────┬─────────┬───┬─┬─────────────────┬──────┬───────────┐
│1 │2 │3 │4 │5 │6 │7│8 │9 │10 │
├───┼─────┼───┼──────────────────────┼─────────┼───┼─┼─────────────────┼──────┼───────────┤
│0 │1 │1 │2 │1 │0 │1│2 │2 │2 │
├───┼─────┼───┼──────────────────────┼─────────┼───┼─┼─────────────────┼──────┼───────────┤
│1 │1 │1 │3 │1 │6 │6│7 │7 │7 │
├───┼─────┼───┼──────────────────────┼─────────┼───┼─┼─────────────────┼──────┼───────────┤
│div│h1 │div│p │p │div│p│ │strong│ │
├───┼─────┼───┼──────────────────────┼─────────┼───┼─┼─────────────────┼──────┼───────────┤
│ │Title│ │nested p in nested div│Some text│ │ │Here is text with│bold │tag inside.│
└───┴─────┴───┴──────────────────────┴─────────┴───┴─┴─────────────────┴──────┴───────────┘

We can now identify nodes whose parent we are going to remove. `remove[p]` says whether our parent is marked for removal. That is, if we are the children of a tag to be removed.

In [14]:
remove[p]

1 1 1 0 1 0 0 1 1 1

`remove∨remove[p]` asks if we or our parents are marked for removal:

In [16]:
remove∨remove[p]

1 1 1 0 1 0 1 1 1 1

To get all descendants, we iterate on this expression until no more children are found.

In [15]:
⎕←desc←{⍵∨⍵[p]}⍣≡remove

1 1 1 1 1 0 1 1 1 1

At this point we could remove the nodes and all their descendants with the *compress* function:

In [10]:
⎕XML ⍉↑(d t v a)⌿¨⍨⊂~desc

<div></div>

But instead we want to keep the descendants and lift them one level in the hierarchy. To do this, we simply take one from the depths of the descendants. Then we remove only those nodes we originally intended to remove.

In [11]:
((desc)⌿d) -← 1
⎕XML ⍉↑(d t v a)⌿¨⍨⊂~remove

<h1>Title</h1>
<div>
 <p>nested p in nested div</p>
</div>
<p>Some text</p>
<div>
 Here is text with
 <strong>bold</strong>
 tag inside.
</div>

We have removed tags with `class="remove"` and lifted their descendants in the tree structure.

In [20]:
d(⊢-1+⍸)p

DOMAIN ERROR: Left argument must be sorted ascending
      d(⊢-1+⍸)p
       ∧


## Garbage collection


## Aaron's notes
- The E function strikes me as somewhat inefficient and probably wouldn't scale well for larger systems.
- When using the D2P idiom on its own, I prefer to put something like {} in front of it or the like to ensure that I am discarding the data from the reduction
- The use of membership to check descendants is quite expensive. There's no need to do this, and the combination of ∊ with ⍸ is even worse. You can instead use the fact that ⍵[p] gives you the Boolean mask of whether your parent is marked for removal or not. Thus, ⍵∨⍵[p] is a more intuitive way to check for removal, and can be read as, "Either we or our parents are marked for removal." 
- You're working in both "depth" and "parent", which is fine to illustrate things, but it might be worth putting some "engineering notes" here that talk about how you wouldn't want to maintain both the depth vector and the parent vector at the same time. I prefer to "go into parent mode" and then do work, and then when I am done with all of that work, go back into "depth mode". 
- You're ending does the removal and the update of the depth vector, but this gives the impression that you don't need to worry about your parent vector. This would lead people down the path of recreating the parent vector on each iteration, which I don't think is warranted. Instead, you might consider introducing the "garbage collection idiom" dm(⊢-1+⍸)pv to show how to update the parent vector appropriately. 
- You also don't show how you might nicely update the columns of the tree, and instead just go straight back into a serialized ⎕XML format. Again, perfectly fine, but if you often want to do more than one thing to a tree, I prefer to use something like d t v a⌿⍨←⊂msk←... as an idiom for filtering things out and updating the columns. This then allows you to chain that together with the collection idiom.

